# Data Quality Validation & Outlier Treatment (TMDB 2010-2025)

This notebook performs **systematic data quality checks** and **outlier treatment** on the TMDB dataset
before it enters the modeling pipeline.

**Sections:**
1. Schema & type validation
2. Missing data analysis & visualization
3. Outlier detection (IQR + z-score methods)
4. Duplicate analysis
5. Domain-specific validation rules
6. Correlation-based feature cleanup
7. Clean dataset export

**Output:** `data/data_quality_validated.csv`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)

%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
# Load the cleaned/engineered dataset
df = pd.read_csv("../data/data_cleaned_engineered.csv")
print(f"Original shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print(f"Rows: {len(df)}")

---
## 1. Schema & Type Validation

Verify that each column has the expected data type and flag any inconsistencies.

In [ ]:
# Expected types for key columns
expected_types = {
    "movie_id": "numeric",
    "title": "string",
    "release_date": "datetime",
    "runtime": "numeric",
    "popularity": "numeric",
    "vote_average": "numeric",
    "vote_count": "numeric",
    "budget": "numeric",
    "revenue": "numeric",
    "overview": "string",
}

print("=" * 60)
print("SCHEMA VALIDATION REPORT")
print("=" * 60)

issues = []
for col, expected in expected_types.items():
    if col not in df.columns:
        issues.append(f"  MISSING: {col}")
        continue
    
    actual = df[col].dtype
    status = "OK"
    
    if expected == "numeric" and not np.issubdtype(actual, np.number):
        status = f"WARN: expected numeric, got {actual}"
        issues.append(f"  {col}: {status}")
    elif expected == "datetime" and actual == "object":
        # Try to parse
        try:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            status = "FIXED: converted to datetime"
        except:
            status = "WARN: could not convert to datetime"
            issues.append(f"  {col}: {status}")
    
    print(f"  {col:20s} | dtype: {str(actual):15s} | {status}")

if issues:
    print(f"\n{len(issues)} issue(s) found:")
    for issue in issues:
        print(issue)
else:
    print("\nAll schema checks passed.")

---
## 2. Missing Data Analysis

Visualize missing data patterns to understand if data is MCAR, MAR, or MNAR.

In [ ]:
# Missing data summary
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_report = pd.DataFrame({
    "missing_count": missing,
    "missing_pct": missing_pct
}).sort_values("missing_pct", ascending=False)

# Show only columns with missing values
missing_report = missing_report[missing_report["missing_count"] > 0]
print(f"Columns with missing values: {len(missing_report)} / {len(df.columns)}")
print(missing_report)

In [ ]:
# Visualize missing data patterns
if len(missing_report) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar chart of missing percentages
    top_missing = missing_report.head(15)
    axes[0].barh(top_missing.index, top_missing["missing_pct"], color="coral")
    axes[0].set_xlabel("Missing %")
    axes[0].set_title("Top 15 Columns by Missing Data %")
    axes[0].invert_yaxis()
    
    # Missing data heatmap (sample for readability)
    cols_with_missing = missing_report.head(10).index.tolist()
    if cols_with_missing:
        sample = df[cols_with_missing].sample(min(200, len(df)), random_state=42)
        sns.heatmap(sample.isnull(), cbar=True, yticklabels=False, ax=axes[1],
                    cmap="YlOrRd")
        axes[1].set_title("Missing Data Pattern (sample of rows)")
    
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found.")

In [ ]:
# Missing data correlation: are columns missing together?
cols_with_missing = df.columns[df.isnull().any()].tolist()

if len(cols_with_missing) > 1:
    missing_corr = df[cols_with_missing].isnull().corr()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(missing_corr, annot=True, fmt=".2f", cmap="coolwarm",
                center=0, ax=ax, vmin=-1, vmax=1)
    ax.set_title("Missing Data Correlation Matrix")
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated missing patterns
    high_corr = []
    for i in range(len(missing_corr.columns)):
        for j in range(i+1, len(missing_corr.columns)):
            if abs(missing_corr.iloc[i, j]) > 0.5:
                high_corr.append((missing_corr.columns[i], missing_corr.columns[j], missing_corr.iloc[i, j]))
    
    if high_corr:
        print("Columns that tend to be missing together (corr > 0.5):")
        for c1, c2, corr in high_corr:
            print(f"  {c1} <-> {c2}: {corr:.3f}")
else:
    print("Not enough columns with missing data for correlation analysis.")

---
## 3. Outlier Detection & Treatment

Identify outliers using both **IQR** and **z-score** methods. We apply treatment strategies
appropriate for each variable.

In [ ]:
def detect_outliers_iqr(series, multiplier=1.5):
    """Detect outliers using the IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - multiplier * IQR
    upper = Q3 + multiplier * IQR
    outliers = (series < lower) | (series > upper)
    return outliers, lower, upper

def detect_outliers_zscore(series, threshold=3):
    """Detect outliers using z-score method."""
    z = np.abs((series - series.mean()) / series.std())
    return z > threshold

# Key numeric columns to check for outliers
outlier_cols = ["runtime", "budget", "revenue", "popularity", "vote_average", 
                "vote_count", "director_popularity", "cast_pop_mean"]
outlier_cols = [c for c in outlier_cols if c in df.columns]

print("=" * 70)
print("OUTLIER DETECTION REPORT")
print("=" * 70)

outlier_summary = []
for col in outlier_cols:
    series = df[col].dropna()
    if len(series) == 0:
        continue
    
    iqr_mask, lower, upper = detect_outliers_iqr(series)
    zscore_mask = detect_outliers_zscore(series)
    
    outlier_summary.append({
        "column": col,
        "n_valid": len(series),
        "iqr_outliers": iqr_mask.sum(),
        "iqr_pct": f"{iqr_mask.mean()*100:.1f}%",
        "zscore_outliers": zscore_mask.sum(),
        "zscore_pct": f"{zscore_mask.mean()*100:.1f}%",
        "iqr_lower": f"{lower:.2f}",
        "iqr_upper": f"{upper:.2f}",
        "min": f"{series.min():.2f}",
        "max": f"{series.max():.2f}",
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))

In [ ]:
# Visualize distributions with outlier thresholds
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, col in enumerate(outlier_cols[:8]):
    ax = axes[i]
    series = df[col].dropna()
    
    # Box plot
    bp = ax.boxplot(series, vert=True, patch_artist=True,
                    boxprops=dict(facecolor='lightblue', color='navy'),
                    medianprops=dict(color='red', linewidth=2))
    ax.set_title(col, fontsize=11, fontweight='bold')
    ax.set_ylabel("Value")

# Hide unused subplots
for j in range(len(outlier_cols), len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Box Plots \u2014 Outlier Visualization", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# Apply outlier treatment: cap extreme values at IQR boundaries (winsorization)
# Only applied to columns where capping makes domain sense

cap_columns = ["runtime", "budget", "revenue", "popularity", "vote_count",
               "director_popularity", "cast_pop_mean"]
cap_columns = [c for c in cap_columns if c in df.columns]

cap_report = []
for col in cap_columns:
    before_min, before_max = df[col].min(), df[col].max()
    _, lower, upper = detect_outliers_iqr(df[col].dropna(), multiplier=3.0)  # Use 3x IQR (conservative)
    
    n_capped_low = (df[col] < lower).sum()
    n_capped_high = (df[col] > upper).sum()
    
    df[f"{col}_capped"] = df[col].clip(lower=lower, upper=upper)
    
    cap_report.append({
        "column": col,
        "capped_low": n_capped_low,
        "capped_high": n_capped_high,
        "cap_lower": f"{lower:.0f}",
        "cap_upper": f"{upper:.0f}",
    })

print("Winsorization report (3x IQR):")
print(pd.DataFrame(cap_report).to_string(index=False))

---
## 4. Duplicate Analysis

Check for exact and near-duplicate movies that could bias the model.

In [ ]:
# Exact duplicates by movie_id
if "movie_id" in df.columns:
    exact_dupes = df.duplicated(subset=["movie_id"], keep=False)
    print(f"Exact duplicate movie_ids: {exact_dupes.sum()}")
    if exact_dupes.sum() > 0:
        print(df[exact_dupes][["movie_id", "title", "release_date"]].sort_values("movie_id"))

# Near-duplicates: same title + same year
if "title" in df.columns and "release_year" in df.columns:
    near_dupes = df.duplicated(subset=["title", "release_year"], keep=False)
    n_near = near_dupes.sum()
    print(f"\nNear-duplicates (same title + year): {n_near}")
    if n_near > 0 and n_near < 50:
        print(df[near_dupes][["movie_id", "title", "release_year", "popularity"]]
              .sort_values(["title", "release_year"]).head(20))
    elif n_near >= 50:
        print(f"  (showing first 20)")
        print(df[near_dupes][["movie_id", "title", "release_year", "popularity"]]
              .sort_values(["title", "release_year"]).head(20))

---
## 5. Domain-Specific Validation Rules

Apply business logic checks that catch data quality issues specific to the movie domain.

In [ ]:
print("=" * 60)
print("DOMAIN VALIDATION RULES")
print("=" * 60)

validations = []

# Rule 1: Runtime should be between 1 and 600 minutes
if "runtime" in df.columns:
    bad_runtime = (df["runtime"].notna()) & ((df["runtime"] < 1) | (df["runtime"] > 600))
    validations.append(("Runtime outside 1-600 min", bad_runtime.sum()))

# Rule 2: Vote average should be between 0 and 10
if "vote_average" in df.columns:
    bad_votes = (df["vote_average"].notna()) & ((df["vote_average"] < 0) | (df["vote_average"] > 10))
    validations.append(("Vote average outside 0-10", bad_votes.sum()))

# Rule 3: Budget should not be negative
if "budget" in df.columns:
    neg_budget = (df["budget"].notna()) & (df["budget"] < 0)
    validations.append(("Negative budget", neg_budget.sum()))

# Rule 4: Revenue should not be negative
if "revenue" in df.columns:
    neg_revenue = (df["revenue"].notna()) & (df["revenue"] < 0)
    validations.append(("Negative revenue", neg_revenue.sum()))

# Rule 5: Release year within expected range
if "release_year" in df.columns:
    bad_year = (df["release_year"].notna()) & ((df["release_year"] < 2010) | (df["release_year"] > 2025))
    validations.append(("Release year outside 2010-2025", bad_year.sum()))

# Rule 6: Popularity should be non-negative
if "popularity" in df.columns:
    neg_pop = (df["popularity"].notna()) & (df["popularity"] < 0)
    validations.append(("Negative popularity", neg_pop.sum()))

# Rule 7: ROI sanity check (unrealistically high ROI > 1000x)
if "roi" in df.columns:
    extreme_roi = (df["roi"].notna()) & (df["roi"] > 1000)
    validations.append(("Extreme ROI > 1000x", extreme_roi.sum()))

# Rule 8: Revenue without budget (potential data issue)
if "budget" in df.columns and "revenue" in df.columns:
    rev_no_budget = (df["revenue"].notna()) & (df["revenue"] > 0) & (df["budget"].isna())
    validations.append(("Revenue present but budget missing", rev_no_budget.sum()))

for rule, count in validations:
    status = "PASS" if count == 0 else f"FAIL ({count} rows)"
    icon = "OK" if count == 0 else "!!"
    print(f"  [{icon}] {rule:45s} -> {status}")

---
## 6. Feature Correlation Analysis

Identify highly correlated features that may cause multicollinearity in models.

In [ ]:
# Compute correlation matrix for numeric features
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()

# Find highly correlated pairs (|r| > 0.85)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        r = corr_matrix.iloc[i, j]
        if abs(r) > 0.85:
            high_corr_pairs.append({
                "feature_1": corr_matrix.columns[i],
                "feature_2": corr_matrix.columns[j],
                "correlation": round(r, 3)
            })

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values("correlation", ascending=False)
print(f"Feature pairs with |correlation| > 0.85: {len(high_corr_df)}")
if len(high_corr_df) > 0:
    print(high_corr_df.to_string(index=False))

In [ ]:
# Correlation heatmap for key features
key_features = ["runtime", "popularity", "vote_average", "vote_count", "budget", 
                "revenue", "director_popularity", "cast_pop_mean", "genres_count",
                "keywords_count", "overview_word_count", "roi"]
key_features = [c for c in key_features if c in numeric_df.columns]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(numeric_df[key_features].corr(), annot=True, fmt=".2f",
            cmap="RdBu_r", center=0, ax=ax, vmin=-1, vmax=1,
            square=True, linewidths=0.5)
ax.set_title("Feature Correlation Heatmap (Key Variables)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# Suggest features to drop based on high correlation
# Strategy: for each highly correlated pair, suggest dropping the one with lower
# correlation to the target (popularity)

if "popularity" in numeric_df.columns and len(high_corr_df) > 0:
    target_corr = numeric_df.corr()["popularity"].abs()
    
    drop_candidates = set()
    keep_reasons = []
    
    for _, row in high_corr_df.iterrows():
        f1, f2 = row["feature_1"], row["feature_2"]
        if f1 == "popularity" or f2 == "popularity":
            continue
        
        corr1 = target_corr.get(f1, 0)
        corr2 = target_corr.get(f2, 0)
        
        if corr1 >= corr2:
            drop_candidates.add(f2)
            keep_reasons.append(f"  Keep {f1} (|r|={corr1:.3f}) over {f2} (|r|={corr2:.3f})")
        else:
            drop_candidates.add(f1)
            keep_reasons.append(f"  Keep {f2} (|r|={corr2:.3f}) over {f1} (|r|={corr1:.3f})")
    
    print(f"\nSuggested features to drop ({len(drop_candidates)}):")
    for feat in sorted(drop_candidates):
        print(f"  - {feat}")
    print("\nReasoning:")
    for reason in keep_reasons:
        print(reason)
else:
    print("No highly correlated feature pairs found, or no target column.")

---
## 7. Data Quality Summary & Export

Generate a final quality report and export the validated dataset.

In [ ]:
print("=" * 60)
print("FINAL DATA QUALITY SUMMARY")
print("=" * 60)

total_rows = len(df)
total_cols = len(df.columns)
total_cells = total_rows * total_cols
total_missing = df.isnull().sum().sum()
completeness = (1 - total_missing / total_cells) * 100

print(f"\n  Rows:           {total_rows:,}")
print(f"  Columns:        {total_cols}")
print(f"  Total cells:    {total_cells:,}")
print(f"  Missing cells:  {total_missing:,}")
print(f"  Completeness:   {completeness:.2f}%")

# Duplicate count
if "movie_id" in df.columns:
    n_dupes = df.duplicated(subset=["movie_id"]).sum()
    print(f"  Duplicates:     {n_dupes}")

# Data type breakdown
dtype_counts = df.dtypes.value_counts()
print("\n  Data type breakdown:")
for dtype, count in dtype_counts.items():
    print(f"    {str(dtype):20s}: {count} columns")

print("\n" + "=" * 60)

In [ ]:
# Export validated dataset
out_path = "../data/data_quality_validated.csv"
df.to_csv(out_path, index=False)
print(f"Saved validated dataset to: {out_path}")
print(f"Final shape: {df.shape}")